# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [88]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession
from datetime import timedelta, date
from pyspark.sql.functions import udf, col
from pyspark.sql.types import IntegerType, StringType, StructType, StructField, DateType

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [3]:
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

In [25]:
i94_df = spark.read.parquet("sas_data")

In [103]:
temp_df = spark.read.csv('../../data2/GlobalLandTemperaturesByCity.csv', header=True)

In [40]:
airport_df = spark.read.csv('airport-codes_csv.csv', header=True)

In [21]:
cities_df = spark.read.option("delimiter", ";").csv('us-cities-demographics.csv', header = True)

### Step 2: Explore and Assess the Data
#### Explore the Data & Cleaing the data

In [108]:
i94_df.show(5)

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+-----------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|arrive_date|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+-----------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1| 2016-04-30|
|5748518.0|2016.0|   4.0| 245.0| 438.0|    LOS|2

i94cit, i94port are just code. The origin data is in I94_SAS_Labels_Descriptions.SAS. <br>
Arrdate is SAS date foramt.<br>

In [106]:
i94_df.filter(i94_df.i94cit.isNull()).count()

0

In [80]:
i94_df.filter(i94_df.arrdate.isNull()).count()

0

In [37]:
temp_df.show(5)

+----------+------------------+-----------------------------+-----+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-02-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-03-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
only showing top 5 rows



In [38]:
temp_df.select("Latitude").distinct().show(5)

+--------+
|Latitude|
+--------+
|  36.17N|
|  44.20N|
|  57.05N|
|   4.02S|
|  24.92N|
+--------+
only showing top 5 rows



The data include many countries. I remove other countries without US.

In [99]:
temp_df.select('Country').distinct().show(100)

+--------------------+
|             Country|
+--------------------+
|                Chad|
|              Russia|
|            Paraguay|
|               Yemen|
|             Senegal|
|              Sweden|
|              Guyana|
|               Burma|
|         Philippines|
|             Eritrea|
|            Djibouti|
|            Malaysia|
|           Singapore|
|              Turkey|
|              Malawi|
|                Iraq|
|             Germany|
|         Afghanistan|
|            Cambodia|
|              Jordan|
|              Rwanda|
|               Sudan|
|              France|
|              Greece|
|           Sri Lanka|
|              Taiwan|
|             Algeria|
|   Equatorial Guinea|
|                Togo|
|            Slovakia|
|             Reunion|
|           Argentina|
|             Belgium|
|              Angola|
|             Ecuador|
|               Qatar|
|             Lesotho|
|          Madagascar|
|             Albania|
|             Finland|
|          

In [104]:
temp_df = temp_df.filter(temp_df.Country == 'United States')

In [41]:
airport_df.show(5)

+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|ident|         type|                name|elevation_ft|continent|iso_country|iso_region|municipality|gps_code|iata_code|local_code|         coordinates|
+-----+-------------+--------------------+------------+---------+-----------+----------+------------+--------+---------+----------+--------------------+
|  00A|     heliport|   Total Rf Heliport|          11|       NA|         US|     US-PA|    Bensalem|     00A|     null|       00A|-74.9336013793945...|
| 00AA|small_airport|Aero B Ranch Airport|        3435|       NA|         US|     US-KS|       Leoti|    00AA|     null|      00AA|-101.473911, 38.7...|
| 00AK|small_airport|        Lowell Field|         450|       NA|         US|     US-AK|Anchor Point|    00AK|     null|      00AK|-151.695999146, 5...|
| 00AL|small_airport|        Epps Airpark|         820|       NA|         US|     

I desinged data warehouse, and fact data is immigration data.<br>
This data and fact data do not have and can not extract same key. So I discard this data. (Fact data just have airline. I can not guess airport.)

In [42]:
cities_df.show(5)

+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|            City|        State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race|Count|
+----------------+-------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+-----+
|   Silver Spring|     Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|  Hispanic or Latino|25924|
|          Quincy|Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|               White|58723|
|          Hoover|      Alabama|      38.5|          38040| 

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
sas_date_to_timestamp = udf(lambda x: timedelta(days=x) + date(1960, 1, 1), DateType())
i94_df = i94_df.withColumn('arrive_date', sas_date_to_timestamp(col('arrdate')))

In [141]:
with open('I94_SAS_Labels_Descriptions.SAS') as f:
    descriptions = f.readlines()
    
def extract_code(codes):
    temp = []
    for code in codes:
        code = code.strip().replace("'", '').split('=')
        code = [c.strip() for c in code]
        temp.append(code)
    return pd.DataFrame(temp)

cits = descriptions[10:298]
cit_pd_df = extract_code(cits)
cit_pd_df.columns = ['i94cit', 'cit']
# print(cit_pd_df['i94cit'])
cit_pd_df['i94cit'] = cit_pd_df['i94cit'].astype('int')
schema = StructType([StructField('i94cit', IntegerType(), True),
                     StructField('cit', StringType(), True)])
cit_df = spark.createDataFrame(cit_pd_df,schema=schema)

ports = descriptions[303:962]
port_pd_df = extract_code(ports)
port_pd_df.columns = ['i94port', 'port']

port_pd_df['port_city'] = port_pd_df.port.str.split(',').str[0]
port_pd_df['port_state'] = port_pd_df.port.str.split(',').str[1]
port_pd_df = port_pd_df.drop(columns=['port'])
schema = StructType([StructField('i94port', StringType(), True),
                     StructField('port_city', StringType(), True),
                     StructField('port_state', StringType(), True)])
port_df = spark.createDataFrame(port_pd_df,schema=schema)

addrs = descriptions[982:1036]
addr_pd_df = extract_code(addrs)
addr_pd_df.columns = ['i94addr', 'addr']

schema = StructType([StructField('i94addr', StringType(), True),
                     StructField('addr', StringType(), True)])
addr_df = spark.createDataFrame(addr_pd_df,schema=schema)

In [144]:
i94_df.join(cit_df, 'i94cit') \
    .join(port_df, 'i94port') \
    .join(addr_df, 'i94addr') \
    .select(col('cicid').alias('id'),
            col('arrive_date'),
            col('biryear'),
            col('addr'),
            col('gender'),
            col('admnum'),
            col('visatype'),
            col('i94yr').alias('year'),
            col('i94mon').alias('month'),
            col('cit'),
            col('port_city'),
            col('port_state'),
            col('i94mode').alias('mode'),
            col('i94visa').alias('visa')).show()

+---------+-----------+-------+-------+------+---------------+--------+------+-----+----------+----------+----------+----+----+
|       id|arrive_date|biryear|   addr|gender|         admnum|visatype|  year|month|       cit| port_city|port_state|mode|visa|
+---------+-----------+-------+-------+------+---------------+--------+------+-----+----------+----------+----------+----+----+
|1341668.0| 2016-04-07| 2000.0|ARIZONA|     M| 9.296699883E10|      B2|2016.0|  4.0|   ECUADOR|FORT MYERS|        FL| 1.0| 2.0|
|5439767.0| 2016-04-29| 1967.0|ARIZONA|     M|5.9500084533E10|      WT|2016.0|  4.0|    NORWAY|FORT MYERS|        FL| 1.0| 2.0|
|3386206.0| 2016-04-18| 1962.0|ARIZONA|     M|5.6359932933E10|      WB|2016.0|  4.0| AUSTRALIA|FORT MYERS|        FL| 1.0| 1.0|
|3387636.0| 2016-04-18| 1949.0|ARIZONA|     F|5.6384782833E10|      WT|2016.0|  4.0| AUSTRALIA|FORT MYERS|        FL| 1.0| 2.0|
| 815532.0| 2016-04-05| 1943.0|ARIZONA|     F|5.5688696333E10|      WT|2016.0|  4.0|   DENMARK|FORT MYER

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.